# Imports

In [ ]:
from pathlib import Path
from asapdiscovery.data.openeye import load_openeye_sdfs, oechem, save_openeye_sdfs
from asapdiscovery.data.fragalysis import parse_fragalysis
import numpy as np
import pandas as pd, numpy as np
import plotly.express as px
from datetime import datetime
from tqdm.notebook import tqdm
from asapdiscovery.docking.analysis import DockingResults
import asapdiscovery.data.openeye as oe
import asapdiscovery.modeling.cheminformatics as ci
from importlib import reload

# Load Paths

In [ ]:
import sys
sys.path.append(str(Path("../../../").resolve()))
from software.paths import paths

In [ ]:
local_analysis = Path("/Users/alexpayne/Scientific_Projects/mers-drug-discovery/sars2-retrospective-analysis/")

In [ ]:
full_posit = local_analysis / "results_with_structure_dates.csv"
hybrid = local_analysis / "results_cleaned_tc_scores.csv"

In [ ]:
posit_df = pd.read_csv(full_posit, index_col=0)

In [ ]:
len(posit_df)

In [ ]:
hybrid_df = pd.read_csv(hybrid, index_col=0)

In [ ]:
len(hybrid_df)

# create joined csv

In [ ]:
hybrid_df.columns

In [ ]:
tidy_columns = ["Docked_File", "RMSD", "POSIT", "POSIT_Method", "Chemgauss4", "Clash", "POSIT_R"]

In [ ]:
complex_info_cols = [col for col in hybrid_df.columns if col not in tidy_columns]
hybrid_complex_info = hybrid_df.loc[:, complex_info_cols]
complex_info_cols = [col for col in posit_df.columns if col not in tidy_columns]
posit_complex_info = posit_df.loc[:, complex_info_cols]

In [ ]:
on_cols = [col for col in posit_complex_info.columns if col in hybrid_complex_info.columns]

In [ ]:
complex_info_df = pd.merge(posit_complex_info, hybrid_complex_info, on=on_cols)

In [ ]:
hybrid_to_merge = hybrid_df.loc[:, tidy_columns + ["Complex_ID"]]

In [ ]:
hybrid_to_merge["Version"] = "Hybrid-Only"

In [ ]:
hybrid_to_merge

In [ ]:
posit_to_merge = posit_df.loc[:, tidy_columns + ["Complex_ID"]]

In [ ]:
posit_to_merge["Version"] = "All"

In [ ]:
tidy_df = pd.concat([posit_to_merge, hybrid_to_merge])

In [ ]:
np.shape(tidy_df)

In [ ]:
outer = pd.merge(tidy_df, complex_info_df, how="outer", on="Complex_ID")

In [ ]:
inner = pd.merge(tidy_df, complex_info_df, how="inner", on="Complex_ID")

In [ ]:
np.shape(outer)

In [ ]:
np.shape(inner)

## conclusion: better to use pd.merge(how="outer") to capture all the values

In [ ]:
missing = set(outer.Complex_ID.unique()) - set(inner.Complex_ID.unique())

In [ ]:
gained = outer[outer.Complex_ID.isin(missing)]

In [ ]:
gained.Compound_ID.unique()

### problem: why didn't these compound ids get included in the complex info?

### because i also didn't use inner / outer correctly here, when generating the complex info

In [ ]:
complex_info_df = pd.merge(posit_complex_info, hybrid_complex_info, on=on_cols, how="outer")

In [ ]:
outer = pd.merge(tidy_df, complex_info_df, how="outer", on="Complex_ID")

In [ ]:
inner = pd.merge(tidy_df, complex_info_df, how="inner", on="Complex_ID")

In [ ]:
set(outer.Complex_ID.unique()) - set(inner.Complex_ID.unique())

## now nothing is missing

In [ ]:
len(outer.Reference_Ligand.unique())

In [ ]:
len(outer.Structure_Source.unique())

In [ ]:
len(outer.Compound_ID.unique())

In [ ]:
len(outer.Complex_ID.unique())

In [ ]:
219*219

# Why so many in "ALL"? Can't be more than 47961

In [ ]:
len(posit_df.Complex_ID.unique())

In [ ]:
posit_df.groupby("Compound_ID").apply(lambda x: len(x.Structure_Source.unique())).unique()

In [ ]:
outer.groupby(["Version"])["Complex_ID"].apply(lambda x: len(x.unique()))

In [ ]:
outer.groupby(["Version"])["Complex_ID"].count()

In [ ]:
_ = outer.groupby(["Version", "Complex_ID"])["Complex_ID"].count()

In [ ]:
duplicates = _[_ > 1]

In [ ]:
duplicates["All"].index

In [ ]:
dups_removed = outer.drop_duplicates()

In [ ]:
dups_removed.groupby(["Version"])["Complex_ID"].apply(lambda x: len(x.unique()))

In [ ]:
dups_removed.groupby(["Version"])["Complex_ID"].count()

In [ ]:
_ = dups_removed.groupby(["Version", "Complex_ID"])["Complex_ID"].count()

In [ ]:
duplicates = _[_ > 1]

In [ ]:
duplicates["All"].index

In [ ]:
np.shape(duplicates)

In [ ]:
duplicates

In [ ]:
dups_removed[dups_removed.Complex_ID == "ALP-POS-133e7cd9-2_Mpro-P0010_0A"]

### so we've kept 214 reference structures and 208 query ligands

## save the combined df

In [ ]:
merged.to_csv(local_analysis / "20230610-combined.csv")